In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('video_game.csv')

In [3]:
print(df.shape)
df.head()

(16719, 17)


,Unnamed: 0,Name,Platform,Year_of_Release,Genre,Publisher,NA_players,EU_players,JP_players,Other_players,Global_players,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.columns

Index(['Unnamed: 0', 'Name', 'Platform', 'Year_of_Release', 'Genre',
       'Publisher', 'NA_players', 'EU_players', 'JP_players', 'Other_players',
       'Global_players', 'Critic_Score', 'Critic_Count', 'User_Score',
       'User_Count', 'Developer', 'Rating'],
      dtype='object')

In [5]:
#drop the columns that won't be necessary
df.drop(['Unnamed: 0', 'Critic_Count', 'User_Count', 'Developer'], axis=1, inplace=True)

In [6]:
#rename the columns to their actual description
col_rename = {'NA_players': 'NA_Sales',
              'EU_players': 'EU_Sales',
              'JP_players': 'JP_Sales',
              'Other_players': 'Other_Country_Sales',
              'Platform': 'Console',
              'Rating': 'ESRB_rating'}
df.rename(columns=col_rename, inplace=True)

In [7]:
#check for null values
df.isnull().sum()

Name                      2
Console                   0
Year_of_Release         269
Genre                     2
Publisher                54
NA_Sales                  0
EU_Sales                  0
JP_Sales                  0
Other_Country_Sales       0
Global_players            0
Critic_Score           8582
User_Score             6704
ESRB_rating            6769
dtype: int64

In [8]:
#get rid of null values for some columns you will keep
df = df.loc[df.ESRB_rating.isnull() == False]
df = df.loc[df.Publisher.isnull() == False]

In [9]:
print(df.shape)
df.isnull().sum()

(9943, 13)


Name                      0
Console                   0
Year_of_Release         176
Genre                     0
Publisher                 0
NA_Sales                  0
EU_Sales                  0
JP_Sales                  0
Other_Country_Sales       0
Global_players            0
Critic_Score           1893
User_Score               70
ESRB_rating               0
dtype: int64

In [10]:
df.dtypes

Name                    object
Console                 object
Year_of_Release        float64
Genre                   object
Publisher               object
NA_Sales               float64
EU_Sales               float64
JP_Sales               float64
Other_Country_Sales    float64
Global_players         float64
Critic_Score           float64
User_Score              object
ESRB_rating             object
dtype: object

In [11]:
#replace critc_score with the mean critic_score of games with the same publisher and genre
temp = df.dropna()
#df.groupby(['Genre', 'Publisher']).mean()

In [12]:
#function to get the mean of a certain column given the publisher and game genre
def get_mean_pub(gen, pub, col):
    temp = df.dropna()
    #User_Score of of type object
    if col == 'User_Score':
        #get rid of tbd rows
        temp = temp.loc[temp.User_Score != 'tbd']
        #change type of column
        temp.User_Score = temp.User_score.astype('float64')
    x = temp.groupby(['Genre', 'Publisher']).mean()[col].reset_index()
    return x.loc[(x.Publisher == pub) & (x.Genre == gen)][col].values[0]
    
#get_mean_pub('Action', 'Nintendo', 'Critic_Score')

In [13]:
print(df.shape)
df.head()

(9943, 13)


,Name,Console,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Country_Sales,Global_players,Critic_Score,User_Score,ESRB_rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,8,E
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,8,E
6,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,8.5,E
7,Wii Play,Wii,2006.0,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,6.6,E


In [14]:
#replace the tbd in user_score to nulls and change the type of the columns
df.User_Score = df.User_Score.replace('tbd', None).astype('float64')

In [15]:
#replace null values in critical_score column with the mean of its publisher and genre
for index, row in df.iterrows():
    if np.isnan(row.Critic_Score):
        #some rpublisher might have only one game so it'll break
        try:
            #Replace with mean of publisher and genre
            df.loc[index, 'Critic_Score'] = round(get_mean_pub(row.Genre, row.Publisher, 'Critic_Score'), 1)
        except:
            #drop index if its the only game and genre
            df.drop(index, inplace=True)
            
    elif np.isnan(row.User_Score):
        #some rpublisher might have only one game so it'll break
        try:
            #Replace with mean of publisher and genre
            df.loc[index, 'User_Score'] = round(get_mean_pub(row.Genre, row.Publisher, 'User_Score'), 1)
        except:
            #drop index if its the only game and genre
            df.drop(index, inplace=True)



In [16]:
print(df.shape)
df.isnull().sum()

(9582, 13)


Name                     0
Console                  0
Year_of_Release        169
Genre                    0
Publisher                0
NA_Sales                 0
EU_Sales                 0
JP_Sales                 0
Other_Country_Sales      0
Global_players           0
Critic_Score             0
User_Score              37
ESRB_rating              0
dtype: int64

In [17]:
#drop all other null values
df = df.dropna()

In [18]:
df.shape

(9377, 13)

In [19]:
df.head()

,Name,Console,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Country_Sales,Global_players,Critic_Score,User_Score,ESRB_rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,8.0,E
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,8.0,E
6,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,8.5,E
7,Wii Play,Wii,2006.0,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,6.6,E


In [22]:
#bring other dataset
df1 = pd.read_csv('Managerial_and_Decision_Economics_2013_Video_Games_Dataset.csv', encoding = "ISO-8859-1")

In [23]:
df1.isnull().sum()

Console                0
Title                  0
US Sales (millions)    0
Block4                 0
Block2                 0
Block1                 0
Block0.5               0
YearReleased           0
2004                   0
2005                   0
2006                   0
2007                   0
2008                   0
2009                   0
2010                   0
YearReleasedSq         0
Publisher              0
Genre                  0
Sequel                 0
Re-release             0
Usedprice              0
lnUsedPrice            0
Review Score           0
ReviewSq               0
RatingE                0
RatingT                0
RatingM                0
Lifecycle              0
LifecycleSq            0
MaxPlayers             0
                      ..
Cricket                0
Fishing                0
Cycling                0
Golf                   0
Hockey                 0
Horse                  0
Hunting                0
Motorcycle             0
OffRoad                0


In [24]:
df1.head()

,Console,Title,US Sales (millions),Block4,Block2,Block1,Block0.5,YearReleased,2004,2005,...,Tricks,Volleyball,Wakeboarding,Wrestling,FirstPerson,Platform,Isometric,SideScrolling,TopDown,ThirdPerson
0,Nintendo DS,Super Mario 64 DS,4.69,1,1,1,1,2004,1,0,...,0,0,0,0,0,1,0,0,0,1
1,Sony PSP,Lumines: Puzzle Fusion,0.56,0,0,0,1,2004,1,0,...,0,0,0,0,0,0,0,0,0,0
2,Nintendo DS,WarioWare Touched!,0.54,0,0,0,1,2004,1,0,...,0,0,0,0,0,1,1,1,1,1
3,Sony PSP,Hot Shots Golf: Open Tee,0.49,0,0,0,0,2004,1,0,...,0,0,0,0,0,0,0,0,0,1
4,Nintendo DS,Spider-Man 2,0.45,0,0,0,0,2004,1,0,...,0,0,0,0,0,1,0,1,0,1


In [25]:
df1.columns

Index(['Console', 'Title', 'US Sales (millions)', 'Block4', 'Block2', 'Block1',
       'Block0.5', 'YearReleased', '2004', '2005',
       ...
       'Tricks', 'Volleyball', 'Wakeboarding', 'Wrestling', 'FirstPerson',
       'Platform', 'Isometric', 'SideScrolling', 'TopDown', 'ThirdPerson'],
      dtype='object', length=166)

In [26]:
#grab only necessary columns
df1 = df1[['Title', 'Sequel', 'Lifecycle', 'MaxPlayers', 'Online', 'Licensed', 'Handheld', 'Accessory', 'LtdEdition',
     'Multiplatform','RealTime', 'TurnBased','VisualNovel','FirstPerson', 'SideScrolling','TopDown', 'ThirdPerson']]

In [27]:
df.head()

,Name,Console,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Country_Sales,Global_players,Critic_Score,User_Score,ESRB_rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,8.0,E
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,8.0,E
6,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,8.5,E
7,Wii Play,Wii,2006.0,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,6.6,E


In [28]:
df1.head()

,Title,Sequel,Lifecycle,MaxPlayers,Online,Licensed,Handheld,Accessory,LtdEdition,Multiplatform,RealTime,TurnBased,VisualNovel,FirstPerson,SideScrolling,TopDown,ThirdPerson
0,Super Mario 64 DS,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1
1,Lumines: Puzzle Fusion,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0
2,WarioWare Touched!,1,0,2,0,0,1,0,0,0,0,0,0,0,1,1,1
3,Hot Shots Golf: Open Tee,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,1
4,Spider-Man 2,1,0,1,0,0,1,0,0,1,1,0,0,0,1,0,1


In [29]:
#merge both dataframes
x1 = df.merge(df1, left_on='Name', right_on='Title')
#drop the title column because it is redundant
x1.drop(['Title'], axis=1, inplace=True)

In [30]:
x1.shape

(3457, 29)

In [31]:
d_names = list(x1.Name.values)

In [32]:
x2 = df[~df['Name'].isin(d_names)]

In [33]:
final_df = pd.concat([x1, x2])

In [34]:
final_df.shape

(10921, 29)

In [35]:
final_df.isnull().sum()

Accessory              7464
Console                   0
Critic_Score              0
ESRB_rating               0
EU_Sales                  0
FirstPerson            7464
Genre                     0
Global_players            0
Handheld               7464
JP_Sales                  0
Licensed               7464
Lifecycle              7464
LtdEdition             7464
MaxPlayers             7464
Multiplatform          7464
NA_Sales                  0
Name                      0
Online                 7464
Other_Country_Sales       0
Publisher                 0
RealTime               7464
Sequel                 7464
SideScrolling          7464
ThirdPerson            7464
TopDown                7464
TurnBased              7464
User_Score                0
VisualNovel            7464
Year_of_Release           0
dtype: int64

In [38]:
#fill maxPlayers value with 1
final_df.MaxPlayers.fillna(value=1, inplace=True)

In [41]:
#fill everything else with 0
final_df.fillna(value=0, inplace=True)

In [42]:
final_df.isnull().sum()

Accessory              0
Console                0
Critic_Score           0
ESRB_rating            0
EU_Sales               0
FirstPerson            0
Genre                  0
Global_players         0
Handheld               0
JP_Sales               0
Licensed               0
Lifecycle              0
LtdEdition             0
MaxPlayers             0
Multiplatform          0
NA_Sales               0
Name                   0
Online                 0
Other_Country_Sales    0
Publisher              0
RealTime               0
Sequel                 0
SideScrolling          0
ThirdPerson            0
TopDown                0
TurnBased              0
User_Score             0
VisualNovel            0
Year_of_Release        0
dtype: int64

In [44]:
final_df.dtypes

Accessory              float64
Console                 object
Critic_Score           float64
ESRB_rating             object
EU_Sales               float64
FirstPerson            float64
Genre                   object
Global_players         float64
Handheld               float64
JP_Sales               float64
Licensed               float64
Lifecycle              float64
LtdEdition             float64
MaxPlayers             float64
Multiplatform          float64
NA_Sales               float64
Name                    object
Online                 float64
Other_Country_Sales    float64
Publisher               object
RealTime               float64
Sequel                 float64
SideScrolling          float64
ThirdPerson            float64
TopDown                float64
TurnBased              float64
User_Score             float64
VisualNovel            float64
Year_of_Release        float64
dtype: object

In [45]:
#change Year_of_Release to int
final_df.Year_of_Release = final_df.Year_of_Release.astype('int64')

In [47]:
#that should do it, all thats left is to format it into an order I would like

In [74]:
final_df = final_df[['Name','Console', 'Year_of_Release', 'Publisher', 'ESRB_rating', 'Global_players', 'Genre', 'Critic_Score', 'User_Score',
         'NA_Sales','EU_Sales','JP_Sales','Other_Country_Sales','Sequel', 'Lifecycle', 'MaxPlayers', 'Online', 'Licensed',
       'Handheld', 'Accessory', 'LtdEdition', 'Multiplatform', 'RealTime',
       'TurnBased', 'VisualNovel', 'FirstPerson', 'SideScrolling', 'TopDown',
       'ThirdPerson']]

In [75]:
final_df.head()

,Name,Console,Year_of_Release,Publisher,ESRB_rating,Global_players,Genre,Critic_Score,User_Score,NA_Sales,...,Accessory,LtdEdition,Multiplatform,RealTime,TurnBased,VisualNovel,FirstPerson,SideScrolling,TopDown,ThirdPerson
0,Mario Kart Wii,Wii,2008,Nintendo,E,35.52,Racing,82.0,8.3,15.68,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Wii Sports Resort,Wii,2009,Nintendo,E,32.77,Sports,80.0,8.0,15.61,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,New Super Mario Bros.,DS,2006,Nintendo,E,29.80,Platform,89.0,8.5,11.28,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,Wii Play,Wii,2006,Nintendo,E,28.92,Misc,58.0,6.6,13.96,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,New Super Mario Bros. Wii,Wii,2009,Nintendo,E,28.32,Platform,87.0,8.4,14.44,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [76]:
#some items repeat - groupby to get rid of them and just grab the last instance
final_df = final_df.groupby(['Name', 'Console','Year_of_Release','ESRB_rating']).last().reset_index()

In [77]:
final_df.shape

(9376, 29)

In [78]:
final_df

,Name,Console,Year_of_Release,ESRB_rating,Publisher,Global_players,Genre,Critic_Score,User_Score,NA_Sales,...,Accessory,LtdEdition,Multiplatform,RealTime,TurnBased,VisualNovel,FirstPerson,SideScrolling,TopDown,ThirdPerson
0,Tales of Xillia 2,PS3,2012,T,Namco Bandai Games,0.84,Role-Playing,71.0,7.9,0.20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,.hack//Infection Part 1,PS2,2002,T,Atari,1.27,Role-Playing,75.0,8.5,0.49,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,.hack//Mutation Part 2,PS2,2002,T,Atari,0.68,Role-Playing,76.0,8.9,0.23,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,.hack//Outbreak Part 3,PS2,2002,T,Atari,0.46,Role-Playing,70.0,8.7,0.14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,007 Racing,PS,2000,T,Electronic Arts,0.53,Racing,51.0,4.6,0.30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,007: Quantum of Solace,DS,2008,T,Activision,0.13,Action,65.0,8.7,0.11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,007: Quantum of Solace,PC,2008,T,Activision,0.03,Action,70.0,6.3,0.01,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,007: Quantum of Solace,PS3,2008,T,Activision,1.14,Action,65.0,6.6,0.43,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,007: Quantum of Solace,Wii,2008,T,Activision,0.65,Action,54.0,7.5,0.29,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,007: Quantum of Solace,X360,2008,T,Activision,1.48,Action,65.0,7.1,0.82,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#turn to new csv
data.to_csv('cleaned_video_game.csv', index=False)